In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import transforms3d as t3d
from stl import mesh
import mpl_toolkits.mplot3d as mplot3d

#import cpp_dtw as cdtw

In [ ]:
modeldata_info = np.load("../forkdata/forkTrajectoryData.npz")

In [ ]:
test_data = modeldata_info["test"]
pose_scaling = modeldata_info["pose_scaling"]
pose_mean = modeldata_info["pose_mean"]
unscale_test = (test_data / pose_scaling) + pose_mean

In [ ]:
plt.figure(figsize=(10,10))
for i in range(100):
    ok = test_data[i,:,2] < 0.03
    plt.plot(test_data[i,ok,0].T,test_data[i,ok,1].T,color="gray",alpha=0.2);
plt.axis("equal");

In [ ]:

figure = plt.figure(figsize=(10,10))
axes = plt.axes(projection='3d')
cmap = plt.get_cmap("viridis")
th = np.linspace(0,np.pi*2,200)
radius = 84/1000
center = [0.4,0.7,0.017]
axes.plot(np.sin(th)*radius + center[0], np.cos(th)*radius+center[1],0*th+center[2],c=((0,0,0,0.5)))


for trajid in [0,3,6,8,9,10]: 
    xs = unscale_test[trajid,:,0]
    ys = unscale_test[trajid,:,1]
    zs = unscale_test[trajid,:,2]
    axes.plot(xs,ys,zs,color="k",linewidth=3)


    for i in [0]:#[int(t) for t in np.linspace(0,100-1,5)]:
        if trajid in [0,3,6,8,9,10]:
            translation = np.array((xs[i],ys[i],zs[i])).reshape(3,1)
            quaternion = unscale_test[trajid,i,3:7] 
            #print(quaternion)
            rotmat = t3d.quaternions.quat2mat(quaternion)

            transformmat = np.concatenate((
                              np.concatenate((rotmat,translation),axis=1),
                              np.array((0,0,0,1)).reshape((1,4))),axis=0)
            #print(transformmat)

            # Load the STL files and add the vectors to the plot
            your_mesh = mesh.Mesh.from_file('forkTipFromTurboSquid.stl')

            # convert mm to m
            your_mesh.vectors /= (1000 / 2)

            your_mesh.transform(transformmat)

            polycollection = mplot3d.art3d.Poly3DCollection(your_mesh.vectors)
            polycollection.set_facecolor((.4,.4,.4,0.4))
            polycollection.set_edgecolors((0,0,0,0.05))
            axes.add_collection3d(polycollection)

scale = 0.12
xmid = 0.4
ymid = 0.70
axes.set_xlim(xmid-scale,xmid+scale)
axes.set_ylim(ymid-scale,ymid+scale)
axes.set_zlim(0,2*scale)
axes.set_xlabel("x")
for elev,azim in [(90,0),(10,60)]:
        print("elev,azim: ",elev,azim)
        axes.view_init(elev=elev, azim=azim)
        axes.set_axis_off()
        display(figure);
        figure.savefig(f"example-data-elev{elev}-azim{azim}.png")